<h1><center>Fetch Stock Price & News</center></h1>
<h2><center>Parallel Function Calling </center></h2>
<h3><center>Build AI Apps - Beginner Level</center></h3>

## Before you start

In order to complete the project you will need to create a developer account with OpenAI and store your API key as an environment variable. Instructions for these steps are outlined below.

### Create a developer account with OpenAI

1. Go to the [API signup page](https://platform.openai.com/signup). 

2. Create your account (you'll need to provide your email address and your phone number).

3. Go to the [API keys page](https://platform.openai.com/account/api-keys). 

4. Create a new secret key.


5. **Take a copy of it**. (If you lose it, delete the key and create a new one.)

### Add a payment method

OpenAI sometimes provides free credits for the API, but this can vary based on geography. You may need to add debit/credit card details. 

**Using the `gpt-3.5-turbo` model in this project should incur a cost less than 1 US cent (but if you rerun tasks, you will be charged every time).** For more information on pricing, see [OpenAI's pricing page](https://openai.com/pricing).

1. Go to the [Payment Methods page](https://platform.openai.com/account/billing/payment-methods).

2. Click Add payment method.

3. Fill in your card details.

### Install open ai library

In [64]:
# !pip install openai

### Load Open AI Key

In [65]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
print(find_dotenv())

OPEN_API_KEY=os.getenv('OPENAI_API_KEY')

# print(OPEN_API_KEY)


C:\Users\agaan\openaiapps\.env


### Create Client

In [66]:

from openai import OpenAI

# Define Model
model="gpt-4o-mini"

#define client
client = OpenAI(api_key=OPEN_API_KEY)

## Application

In [67]:
# !pip install yahooquery

In [68]:
from yahooquery import Ticker

In [69]:
# user_prompt1="What is the stock price of TSLA today?"
# # user_prompt1="What is the stock news of TSLA today?"
# # user_prompt1="What is the stock price and news of TSLA today?"

# response=client.chat.completions.create(model=model,
#                               messages=[{"role":"user","content":user_prompt1}
#                                        ]
#                               )

# print(response.choices[0].message.content)

## Define Functions/Tools

### Get Stock Price

In [70]:
def get_stock_price(ticker):
    try:
        # Create a Ticker object for the provided ticker symbol
        t = Ticker(ticker)
        # Retrieve the price data
        price_data = t.price
        # Check if we received valid data for the ticker
        if ticker in price_data and price_data[ticker].get("regularMarketPrice") is not None:
            price = price_data[ticker]["regularMarketPrice"]
        else:
            return f"Price information for {ticker} is unavailable."
    except Exception as e:
        return f"Failed to retrieve data for {ticker}: {str(e)}"
    
    return f"{ticker} is currently trading at ${price:.2f}"

In [71]:
get_stock_price("TSLA")

'TSLA is currently trading at $342.69'

### Get Stock News

In [72]:
# !pip install feedparser -q

In [73]:
ticker="TSLA"
import feedparser

def get_stock_news(ticker):
    # Construct the RSS feed URL for the given ticker.
    rss_url = f"https://news.google.com/rss/search?q={ticker}+stock"
    try:
        feed = feedparser.parse(rss_url)
        if not feed.entries:
            return f"No news found for {ticker}."
        # Extract the top 3 news items (title and link)
        news_items = []
        for entry in feed.entries[:3]:
            title = entry.get("title", "No Title")
            link = entry.get("link", "No Link")
            news_items.append(f"{title} ({link})")
        news_str = "\n".join(news_items)
    except Exception as e:
        return f"Failed to retrieve news for {ticker}: {str(e)}"
    
    return f"Latest news for {ticker}:\n{news_str}"
get_stock_news("TSLA")

"Latest news for TSLA:\nCathie Wood Just Sold Tesla Stock In May But Is She Going Away With The Robotaxi Launch Approaching? - Investor's Business Daily (https://news.google.com/rss/articles/CBMipgFBVV95cUxQN1FsMF9TNE82dWJ6Qy1vaWpHUWN4VEhaRnlNQWx0eGZaT1BvUzZKWUt0Y0hmYjUtXzVXRGtuN0tlal9XcmdoYVk4dzRXT1lES2gwbDh0VG1QaTVrcnhuMWRJbjNpTmRKWEZmVmRqMWlCbkw1eHk1Y25OcHo2czQxZW9GZUJwR3ZCWjZsX1lnZWhBY19WUWs5TGRLUHIxbFlwT0ROdnBR?oc=5)\nWall Street Analysts Clash over Tesla Stock (TSLA) as Robotaxi Launch Nears - TipRanks (https://news.google.com/rss/articles/CBMipgFBVV95cUxNaENRdHJnOUVIcW1KYktmQ3Foc0xmWFNNdlQwVml2eGVOVEFjdUMzaGdfbDBQVXZJN04wSWlORkszdUxyN25XWk41UlR6SlRTYm5RUmJoRUl2ekJSMi1pR0RLUGNHUVNybWJiRkVCNi1hb053YVdQd1R1c2h4cWhDSFJwNkFjNUVRNFI0WEU5bk9kOWY3WTU4UllleDhjcWt4R1dPM25R?oc=5)\nTesla Robotaxi Nearing Launch: Buy, Hold or Sell the Stock Now? - Yahoo Finance (https://news.google.com/rss/articles/CBMihgFBVV95cUxOeEtZanQxdUZxdHVXYzV4VlJFLVc0NU1GRlhVRnBJc3NSMTdwNGwtZ2pLaEY1c1NMNkkyb1JTUUpiRk

## Tool Definition

In [74]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": "Get current stock price for a provided ticker symbol using yahooquery.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string"}
                },
                "required": ["ticker"],
                "additionalProperties": False
            },
            "strict": True
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_stock_news",
            "description": "Get the latest news for a provided ticker symbol by parsing the Google News RSS feed using feedparser.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string"}
                },
                "required": ["ticker"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
]

In [75]:
# user_prompt1="What is the stock price of TSLA today?"
# user_prompt1="What is the stock news of TSLA in India today?"
user_prompt1="What is the stock price and news of TSLA today?"
messages=[{"role":"user","content":user_prompt1}]                  
response=client.chat.completions.create(model=model,
                              messages=messages,
                              tools=tools
                              )



In [76]:
assistant_msg =response.choices[0].message
assistant_msg

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ROdLjNpWCmiRin85iwPkLCnN', function=Function(arguments='{"ticker": "TSLA"}', name='get_stock_price'), type='function'), ChatCompletionMessageToolCall(id='call_2bH4Jdx73MKFoBKE3zpQrhA0', function=Function(arguments='{"ticker": "TSLA"}', name='get_stock_news'), type='function')])

In [77]:
tool_calls=assistant_msg.tool_calls
if tool_calls:
    messages.append({"role":"assistant","tool_calls":[tc.model_dump() for tc in tool_calls]})
# messages

In [78]:
for tool_call in tool_calls:
    func_name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    if func_name == "get_stock_price":
        result = get_stock_price(args["ticker"])
    elif func_name == "get_stock_news":
        result = get_stock_news(args["ticker"])
    else:
        result = "Unknown function"

    # Step 3: Append tool result
    messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": json.dumps(result)
    })

In [79]:
messages

[{'role': 'user',
  'content': 'What is the stock price and news of TSLA today?'},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_ROdLjNpWCmiRin85iwPkLCnN',
    'function': {'arguments': '{"ticker": "TSLA"}', 'name': 'get_stock_price'},
    'type': 'function'},
   {'id': 'call_2bH4Jdx73MKFoBKE3zpQrhA0',
    'function': {'arguments': '{"ticker": "TSLA"}', 'name': 'get_stock_news'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_ROdLjNpWCmiRin85iwPkLCnN',
  'content': '"TSLA is currently trading at $342.69"'},
 {'role': 'tool',
  'tool_call_id': 'call_2bH4Jdx73MKFoBKE3zpQrhA0',
  'content': '"Latest news for TSLA:\\nCathie Wood Just Sold Tesla Stock In May But Is She Going Away With The Robotaxi Launch Approaching? - Investor\'s Business Daily (https://news.google.com/rss/articles/CBMipgFBVV95cUxQN1FsMF9TNE82dWJ6Qy1vaWpHUWN4VEhaRnlNQWx0eGZaT1BvUzZKWUt0Y0hmYjUtXzVXRGtuN0tlal9XcmdoYVk4dzRXT1lES2gwbDh0VG1QaTVrcnhuMWRJbjNpTmRKWEZmVmRqMWlCbkw1eHk1Y25OcHo2czQxZW9GZU

In [82]:
final_response = client.chat.completions.create(
    model=model,
    messages=messages,
)
final_response

ChatCompletion(id='chatcmpl-BeEhAgIxxNpozMHKocP9oUux7Psqi', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="As of today, the stock price of Tesla (TSLA) is **$342.69**.\n\n### Latest News:\n1. **Cathie Wood Just Sold Tesla Stock In May** - Investor's Business Daily discusses whether she is distancing herself as the robotaxi launch approaches. [Read more](https://news.google.com/rss/articles/CBMipgFBVV95cUxQN1FsMF9TNE82dWJ6Qy1vaWpHUWN4VEhaRnlNQWx0eGZaT1BvUzZKWUt0Y0hmYjUtXzVXRGtuN0tlal9XcmdoYVk4dzRXT1lES2gwbDh0VG1QaTVrcnhuMWRJbjNpTmRKWEZmVmRqMWlCbkw1eHk1Y25OcHo2czQxZW9GZUJwR3ZCWjZsX1lnZWhBY19WUWs5TGRLUHIxbFlwT0ROdnBR?oc=5)\n  \n2. **Wall Street Analysts Clash over Tesla Stock (TSLA)** - TipRanks reports on divisions among analysts as the robotaxi launch approaches. [Read more](https://news.google.com/rss/articles/CBMipgFBVV95cUxNaENRdHJnOUVIcW1KYktmQ3Foc0xmWFNNdlQwVml2eGVOVEFjdUMzaGdfbDBQVXZJN04wSWlORkszdUxyN25XWk41UlR6SlRTYm5RUmJoRUl2

In [83]:
from IPython.display import display, Markdown
Markdown(final_response.choices[0].message.content)

As of today, the stock price of Tesla (TSLA) is **$342.69**.

### Latest News:
1. **Cathie Wood Just Sold Tesla Stock In May** - Investor's Business Daily discusses whether she is distancing herself as the robotaxi launch approaches. [Read more](https://news.google.com/rss/articles/CBMipgFBVV95cUxQN1FsMF9TNE82dWJ6Qy1vaWpHUWN4VEhaRnlNQWx0eGZaT1BvUzZKWUt0Y0hmYjUtXzVXRGtuN0tlal9XcmdoYVk4dzRXT1lES2gwbDh0VG1QaTVrcnhuMWRJbjNpTmRKWEZmVmRqMWlCbkw1eHk1Y25OcHo2czQxZW9GZUJwR3ZCWjZsX1lnZWhBY19WUWs5TGRLUHIxbFlwT0ROdnBR?oc=5)
  
2. **Wall Street Analysts Clash over Tesla Stock (TSLA)** - TipRanks reports on divisions among analysts as the robotaxi launch approaches. [Read more](https://news.google.com/rss/articles/CBMipgFBVV95cUxNaENRdHJnOUVIcW1KYktmQ3Foc0xmWFNNdlQwVml2eGVOVEFjdUMzaGdfbDBQVXZJN04wSWlORkszdUxyN25XWk41UlR6SlRTYm5RUmJoRUl2ekJSMi1pR0RLUGNHUVNybWJiRkVCNi1hb053YVdQd1R1c2h4cWhDSFJwNkFjNUVRNFI0WEU5bk9kOWY3WTU4UllleDhjcWt4R1dPM25R?oc=5)

3. **Tesla Robotaxi Nearing Launch: Buy, Hold or Sell the Stock Now?** - Yahoo Finance explores strategies around Tesla's upcoming robotaxi services. [Read more](https://news.google.com/rss/articles/CBMihgFBVV95cUxOeEtZanQxdUZxdHVXYzV4VlJFLVc0NU1GRlhVRnBJc3NSMTdwNGwtZ2pLaEY1c1NMNkkyb1JTUUpiRkV2T1VSUU5McG90Zy0tMVR4TFc2RFdES0RqQWZvN1BWUDFwZGV1MTNyUTBqYXppZjN2bnVYNmwtdGZxMjZpWlVtWXdTZw?oc=5) 

If you need more detailed information or analysis on any specific topic, feel free to ask!